In [2]:
%matplotlib inline
import pandas
from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
import torch
import numpy as np
from os import walk
from os import path
import os
import matplotlib.pyplot as nlp

In [3]:
parasitized = np.empty([13132,3,75,75])
count=0
for root,dirs,names in walk(r'True_parasitized'):
    for name in names:
        img_dir = path.join(root, name)
        img = Image.open(img_dir).resize((75,75))
        parasitized[count]=np.transpose(np.array(img),(2,0,1))/255
        count=count+1
        if count%1000==0:
            print(count)
        #np.append(parasitized, [np.array(img)], axis=0)
print(parasitized.shape)
non_parasitized = np.empty([13029,3,75,75])
count=0
for root,dirs,names in walk(r'True_uninfected'):
    for name in names:
        img_dir = path.join(root, name)
        img = Image.open(img_dir).resize((75,75))
        non_parasitized[count]=np.transpose(np.array(img), (2,0,1))/255
        count=count+1
        if count%1000==0:
            print(count)
        #np.append(parasitized, [np.array(img)], axis=0)
print(non_parasitized.shape)


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
(13132, 3, 75, 75)
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
(13029, 3, 75, 75)


In [28]:
class trainnData(Dataset):
    def __init__(self):
        print('Initialized')
        #self.parasitized=parasitized
        #self.non_parasitized=non_parasitized
        self.p_n=len(parasitized)
        self.n_n=len(non_parasitized)
        print('Done init')
    def __len__(self):
        return 26161
    def __getitem__(self, idx):
        images=np.empty([1,3,75,75])
        labels=np.zeros([1,2])
        
        if idx<self.p_n:
            images[0]=parasitized[idx]
            labels[0][0]=1
        else:
            images[0]=non_parasitized[idx-self.p_n]
            labels[0][1]=1
            
        return [torch.from_numpy(images).double().to('cuda'), torch.from_numpy(labels).double().to('cuda')]

In [2]:
all_dirs = np.empty([len(os.listdir(r'True_uninfected'))+len(os.listdir(r'True_parasitized')),2],dtype=np.chararray)
i=0
for root,dirs,names in walk(r'True_uninfected'):
    for name in names:
        img_dir = os.path.join(root,name)
        all_dirs[i][0]=img_dir
        all_dirs[i][1]=0
        i=i+1
        if i%1000==0:
            print(f"[{i}/{len(all_dirs)}]")
for root,dirs,names in walk(r'True_parasitized'):
    for name in names:
        img_dir = os.path.join(root,name)
        all_dirs[i][0]=img_dir
        all_dirs[i][1]=1
        i=i+1
        if i%1000==0:
            print(f"[{i}/{len(all_dirs)}]")

[1000/26161]
[2000/26161]
[3000/26161]
[4000/26161]
[5000/26161]
[6000/26161]
[7000/26161]
[8000/26161]
[9000/26161]
[10000/26161]
[11000/26161]
[12000/26161]
[13000/26161]
[14000/26161]
[15000/26161]
[16000/26161]
[17000/26161]
[18000/26161]
[19000/26161]
[20000/26161]
[21000/26161]
[22000/26161]
[23000/26161]
[24000/26161]
[25000/26161]
[26000/26161]


In [5]:
class trainingData(Dataset):
    def __init__(self, feature_transform=lambda x:x, target_transform=lambda x:x):
        self.transform=feature_transform
        self.target_transform=target_transform
        #gonna try loading everything lazily, so ima skip init
        print('')
    def __len__(self):
        return len(os.listdir(r'True_uninfected'))+len(os.listdir(r'True_parasitized'))
    def __getitem__(self, idx):
        selected_dirs=[all_dirs[idx][0]]
        n=len(selected_dirs)
        
        images=np.empty([n,3,100,100])
        labels=np.zeros([n,2])
        for i in range(n):
            img=Image.open(r''+str(selected_dirs[i])).resize((100,100))
            images[i]=np.transpose(np.array(img),(2,0,1))/255
            #labels[i]=all_dirs[idx][1]
            if all_dirs[idx][1]==0:
                labels[i][0]=1
            else:
                labels[i][1]=1
        return [torch.from_numpy(images).double().to('cuda'), torch.from_numpy(labels).double().to('cuda')]

In [29]:
dataset = trainnData()
dataload = DataLoader(dataset, batch_size=64, shuffle=True)
vals=(next(iter(dataload)))

Initialized
Done init


In [97]:
from torch import nn
class nett(nn.Module):
    def __init__(self):
        super(nett, self).__init__()
        self.layers=nn.Sequential(
            nn.Conv2d(3,16,7,stride=1,padding=2),
            nn.ReLU(),
            
            #nn.Conv2d(32,50,3,stride=1),
            #nn.ReLU(),
            
            nn.Conv2d(16,10,5,stride=1),
            #nn.BatchNorm2d(4),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            nn.Conv2d(10,5,3, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            nn.Flatten(),
            nn.Linear(1280, 1000),
            nn.ReLU(),
            nn.Linear(1000,2)
        )
        self.conv1=nn.Conv2d(3,16,7,stride=1,padding=2)
        self.conv2=nn.Conv2d(16,10,5,stride=1)
        self.conv3=nn.Conv2d(10,5,3, stride=1)
        self.linear1=nn.Linear(1280, 1000)
        self.linear2=nn.Linear(1000,2)
        self.maxpool= nn.MaxPool2d(2)
        self.relu=nn.ReLU()
        self.flat=nn.Flatten()
        self.batchnorm1=nn.BatchNorm2d(16)
        self.batchnorm2=nn.BatchNorm2d(10)
        self.batchnorm3=nn.BatchNorm2d(5)
    def forward(self, x):
        x=x.double()
        x=self.conv1(x)
        x=self.batchnorm1(x)
        x=self.relu(x)
        x=self.conv2(x)
        x=self.batchnorm2(x)
        x=self.relu(x)
        x=self.maxpool(x)
        x=self.conv3(x)
        x=self.batchnorm3(x)
        x=self.relu(x)
        x=self.maxpool(x)
        x=self.flat(x)
        x=self.linear1(x)
        x=self.relu(x)
        x=self.linear2(x)
        return x

In [98]:
net = nett()
net=net.double()
#vals[0][30][0] = vals[0][30][0].double()
#result = (net(vals[0][30][0].double()))
#result

In [131]:
epochs=1000
lr=0.001
batch_size=64
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=lr)


In [100]:
out_data=np.zeros([1000])

In [101]:
def train_loop(model, train_loader,lr,batch_size,loss_fn,optimizer,count):
    size = len(train_loader)
    for batch, (X,y) in enumerate(train_loader):
        results = model(X.squeeze())
        loss = loss_fn(results,y.squeeze())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch%100==0:
            
            print(f"{count*5+(batch+100)/100},{loss.item()}")
    return loss.item()

In [102]:
net.to('cuda')

nett(
  (layers): Sequential(
    (0): Conv2d(3, 16, kernel_size=(7, 7), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): Conv2d(16, 10, kernel_size=(5, 5), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(10, 5, kernel_size=(3, 3), stride=(1, 1))
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Flatten(start_dim=1, end_dim=-1)
    (9): Linear(in_features=1280, out_features=1000, bias=True)
    (10): ReLU()
    (11): Linear(in_features=1000, out_features=2, bias=True)
  )
  (conv1): Conv2d(3, 16, kernel_size=(7, 7), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(16, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(10, 5, kernel_size=(3, 3), stride=(1, 1))
  (linear1): Linear(in_features=1280, out_features=1000, bias=True)
  (linear2): Linear(in_features=1000, out_features=2, bias=True)
  (maxpool): MaxPool2d(kernel_size=2, stride

In [132]:
for e in range(0,epochs):
    out_data[e]=train_loop(net,dataload,lr,batch_size,loss_fn,optimizer,e) #

1.0,0.0010917062754263805
2.0,0.0029411950877965263
3.0,0.00227035591267906
4.0,0.007144754943592663
5.0,0.007296214746985474
6.0,0.0014139966419389703
7.0,0.0019017640713512142
8.0,0.004408552049812862
9.0,0.0009430263775224798
10.0,0.05346342323372619
11.0,0.01991451597133341
12.0,0.0009142409909899882
13.0,0.02941274515416649
14.0,0.0022685191857200557
15.0,0.00253620965202662
16.0,0.0045669387433878
17.0,0.03974288985339317
18.0,0.008141173461544922
19.0,0.0031370315352338527
20.0,0.0047061482621655
21.0,0.011720671381057363
22.0,0.005744028921929743
23.0,0.0043590013710869836
24.0,0.003132001487793257
25.0,0.022061614466305442
26.0,0.021590749673581072
27.0,0.010300921543649571
28.0,0.003451241678479527
29.0,0.007016734017880446
30.0,0.010452959741050908
31.0,0.020934380143118055
32.0,0.022722705796183056
33.0,0.002804767433012645
34.0,0.021924671079575513
35.0,0.010384770229564367
36.0,0.0037166122217841663
37.0,0.0017426576297444461
38.0,0.007947044060073447
39.0,0.0085144521299

KeyboardInterrupt: 

In [128]:
if torch.argmax(net(torch.from_numpy(np.expand_dims(parasitized[12000],0)).to('cuda'))).item()==0:
    print('infected')
else:
    print('Uninfected')

infected


In [133]:
torch.save(net.state_dict(), 'FirstWorkingModel.pth')